# Introduction to DataFrames
**[Bogumił Kamiński](http://bogumilkaminski.pl/about/), January 3, 2019**

In [1]:
using DataFrames

## Possible pitfalls

### Know what is copied when creating a `DataFrame`

In [2]:
x = DataFrame(rand(3, 5))

,x1,x2,x3,x4,x5
,Float64,Float64,Float64,Float64,Float64
1,0.915031,0.85272,0.553761,0.80879,0.81124
2,0.226544,0.059366,0.217132,0.902919,0.769796
3,0.255446,0.0781189,0.509501,0.79382,0.471198


In [3]:
y = convert(DataFrame, x)
x === y # no copyinng performed

true

In [4]:
y = copy(x)
x === y # not the same object

false

In [5]:
y = DataFrame(x)
x === y

false

In [6]:
all(x[i] === y[i] for i in ncol(x)) # but the columns are the same

true

In [7]:
x = 1:3; y = [1, 2, 3]; df = DataFrame(x=x,y=y) # the same when creating arrays or assigning columns, except ranges

,x,y
,Int64,Int64
1,1,1
2,2,2
3,3,3


In [8]:
y === df[:y] # the same object

true

In [9]:
typeof(x), typeof(df[:x]) # range is converted to a vector

(UnitRange{Int64}, Array{Int64,1})

In [10]:
y === df[:, :y] # slicing rows always creates a copy

false

### Do not modify the parent of `GroupedDataFrame` or `view`

In [11]:
x = DataFrame(id=repeat([1,2], outer=3), x=1:6)
g = groupby(x, :id)

GroupedDataFrame{DataFrame} with 2 groups based on key: :id
First Group (3 rows): :id = 1
│ Row │ id    │ x     │
│     │ Int64 │ Int64 │
├─────┼───────┼───────┤
│ 1   │ 1     │ 1     │
│ 2   │ 1     │ 3     │
│ 3   │ 1     │ 5     │
⋮
Last Group (3 rows): :id = 2
│ Row │ id    │ x     │
│     │ Int64 │ Int64 │
├─────┼───────┼───────┤
│ 1   │ 2     │ 2     │
│ 2   │ 2     │ 4     │
│ 3   │ 2     │ 6     │

In [12]:
x[1:3, 1]=[2,2,2]
g # well - it is wrong now, g is only a view

GroupedDataFrame{DataFrame} with 2 groups based on key: :id
First Group (3 rows): :id = 2
│ Row │ id    │ x     │
│     │ Int64 │ Int64 │
├─────┼───────┼───────┤
│ 1   │ 2     │ 1     │
│ 2   │ 2     │ 3     │
│ 3   │ 1     │ 5     │
⋮
Last Group (3 rows): :id = 2
│ Row │ id    │ x     │
│     │ Int64 │ Int64 │
├─────┼───────┼───────┤
│ 1   │ 2     │ 2     │
│ 2   │ 2     │ 4     │
│ 3   │ 2     │ 6     │

In [13]:
s = view(x, 5:6, :)

,id,x
,Int64,Int64
1,1,5
2,2,6


In [14]:
deleterows!(x, 3:6)

,id,x
,Int64,Int64
1,2,1
2,2,2


In [15]:
s # error

BoundsError: BoundsError: attempt to access 2-element Array{Int64,1} at index [5:6]

### Remember that you can filter columns of a `DataFrame` using booleans

In [16]:
using Random
Random.seed!(1)
x = DataFrame(rand(5, 5))

,x1,x2,x3,x4,x5
,Float64,Float64,Float64,Float64,Float64
1,0.236033,0.210968,0.555751,0.209472,0.0769509
2,0.346517,0.951916,0.437108,0.251379,0.640396
3,0.312707,0.999905,0.424718,0.0203749,0.873544
4,0.00790928,0.251662,0.773223,0.287702,0.278582
5,0.488613,0.986666,0.28119,0.859512,0.751313


In [17]:
x[x[:x1] .< 0.25] # well - we have filtered columns not rows by accident as you can select columns using booleans

,x1,x4
,Float64,Float64
1,0.236033,0.209472
2,0.346517,0.251379
3,0.312707,0.0203749
4,0.00790928,0.287702
5,0.488613,0.859512


In [18]:
x[x[:x1] .< 0.25, :] # probably this is what we wanted

,x1,x2,x3,x4,x5
,Float64,Float64,Float64,Float64,Float64
1,0.236033,0.210968,0.555751,0.209472,0.0769509
2,0.00790928,0.251662,0.773223,0.287702,0.278582


### Column selection for DataFrame creates aliases unless explicitly copied

In [19]:
x = DataFrame(a=1:3)
x[:b] = x[1] # alias
x[:c] = x[:, 1] # copy
x[:d] = x[1][:] # copy
x[:e] = copy(x[1]) # explicit copy
display(x)
x[1,1] = 100
display(x)

,a,b,c,d,e
,Int64,Int64,Int64,Int64,Int64
1,1,1,1,1,1
2,2,2,2,2,2
3,3,3,3,3,3


,a,b,c,d,e
,Int64,Int64,Int64,Int64,Int64
1,100,100,1,1,1
2,2,2,2,2,2
3,3,3,3,3,3
